In [13]:
import pandas as pd
import folium
import branca.colormap as cm
import json

###################################################################################################################################
'''Funktion zum erstellen einer Folium Karte mit allen Voreinstellungen
Übergabewerte sind die Geodaten mit den Landesgrenzen, die kreisliste mit den Datenreihen, die Anzahl der Farbabstufungen, 
die Skalenwerte bei denen eine Abstufung erfolgen soll und das Argument auf das gemappt werden soll'''
def getKarte(geoDaten, kreisliste, colorstep, key_on ):

    #Kreisnummern so verändern, dass sie mit den GoJson Nummern entsprechen
    kreisliste = kreisliste.astype({'kreisnummer' : 'float'})       #get rid of 0 in front
    kreisliste = kreisliste.astype({'kreisnummer' : 'int'})       #get rid of 0 in front
    kreisliste = kreisliste.astype({'kreisnummer' : 'str'})       #only strings can be verglichen werden to show results
    #kreisliste['kreisnummer'] = kreisliste['kreisnummer'] + '000'         #add 000

    #First determine your maximum and minimum values - this becomes the basis for the steps in the map
    vmax = kreisliste[key_on].max()  
    vmin = kreisliste[key_on].min() 
    #Farbverlauf einstellen -> von weiß nach blau zu Lila
    colormap = cm.LinearColormap(colors=['#ffffff', '#61abff', '#b894ff', '#a200ff', '#d000ff'], 
                             vmin=vmin,
                             vmax=vmax)

    #rückgabe des höchsten und mittelwerts in der Tabelle 
    print("max : " +str(vmax))
    print("mean: " +str(kreisliste[key_on].mean()))

    #colormap so einstellen, dass überhang besteht. so werden die Daten von kleineren Landkreisen sichtbar
    colormap = colormap.to_step(len(colorstep),
                data= colorstep,
                method='quantiles',
                round_method='float')

    #karte erstellen - mittelpunkt ist mittelpunkt von deutschland
    map = folium.Map(location = [51.1657,10.4515], zoom_start=6.3)

    #Choropleth karte auf Karte auflegen
    cp = folium.Choropleth(
        geo_data=geoDaten,  #Daten für Landkreisgrenzen
        name="choropleth",  
        data= kreisliste,   #kreisliste als Datensatz pbergeben
        columns=["kreisnummer", key_on],    #zeile kreisnummern zum vergleich mit GeoJson und key_on anzuzeigende Werte
        key_on = 'properties.SN_KRG',       #Ort der Kreisnummern in GeoJson
        line_weight = 0.1,
        fill_opacity= 0.1,
    ).add_to(map)

    #remove one of the two legends
    cp._children
    for key in cp._children:
        if key.startswith('color_map'):
            del(cp._children[key])
        
    #add new legend
    colormap.add_to(map)
    #schreiben der daten in die GEOJSON Datei für tooltips und zum abrufen der Werte für die Colormap
    for s in cp.geojson.data['features']:   #iterieren durch alle json features
        try: 
            row = kreisliste.loc[s['properties']['SN_KRG'] == kreisliste['kreisnummer']] #row index finden wo kreisnummer und SNKRG matcht
            s['properties'][key_on] = str(kreisliste.at[row.index[0], key_on]) #schreiben in die JSON
        except: pass

    #print(cp.geojson.data)
    with open('example.json', 'w') as file:
    # Write the text stored in the variable to the file
        json.dump(cp.geojson.data, file, indent=2)
    #set style 
    highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

    style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.7, 
                            'weight': 0.2,
                            'fillColor': colormap(float(x['properties'][key_on]))       #essentieller part -  ruft colormap mit wert auf
                            }

    #Zusammenfügen aller zuvor festegelgten Personalisierungen 
    NIL = folium.features.GeoJson(
        geoDaten,
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(         #tooltops sind zuvor in GeoJson List geschrieben worden
            fields=['KRG', 'SN_KRG',key_on],            #angezeigt werden kreisnamen, Kreisnummern und der Wert von key_on
            aliases=['Kreisname: ', 'Kreisnummer: ', key_on],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )
           

    map.add_child(NIL)

    return map

In [20]:
import psycopg2
import geoplotlib
from geoplotlib.utils import BoundingBox, DataAccessObject
from geoplotlib.colors import ColorMap
from pandas import DataFrame
import json
import pandas as pd

# Replace with your database credentials
db_connection = psycopg2.connect(
    user="tester",
    password="1234",
    host="localhost",
    port="5432",
    database="ThesisTry1"
)

#updated query to add anzahlmodule to extract Q
query = """
SELECT
    we."EinheitMastrNummer",
    we."Landkreis",
    SUBSTRING(we."Gemeindeschluessel" FROM 1 FOR 2) AS Kreisnummer,
    we."Bruttoleistung"
FROM 
    wind_extended As we
WHERE
    SUBSTRING(CAST(we."Registrierungsdatum" AS TEXT) FROM 1 FOR 4) = '2022'
GROUP BY 
    we."EinheitMastrNummer", we."Landkreis", SUBSTRING(we."Gemeindeschluessel" FROM 1 FOR 2), we."Bruttoleistung";
"""
# Create a cursor and execute the query
cursor = db_connection.cursor()
cursor.execute(query)

# Fetch all the rows from the result
results = cursor.fetchall()

kreisliste = pd.DataFrame(results)
kreisliste.columns = ['EinheitMastrNummer', 'landkreis','kreisnummer','Bruttoleistung']

# Drop rows with missing longitude or latitude values
kreisliste = kreisliste.dropna()
display(kreisliste)

,EinheitMastrNummer,landkreis,kreisnummer,Bruttoleistung
0,SEE900026085282,Dithmarschen,01,6800
1,SEE900037558577,Dithmarschen,01,2990
2,SEE900052108511,Ammerland,03,2300
3,SEE900077931491,Paderborn,05,3000
5,SEE900189117318,Lippe,05,5500
...,...,...,...,...
1626,SEE999784885586,Uckermark,12,5700
1627,SEE999823070461,Rhein-Erft-Kreis,05,3600
1628,SEE999825467962,Ostprignitz-Ruppin,12,6800
1629,SEE999916059649,Spree-Neiße,12,5600


In [22]:
import numpy as np
import pandas as pd

# Group by 'Landkreis' and 'kreisnummer', then calculate the sum of 'brutoleistung'
sum_bruttoleistung_per_kreisnummer = kreisliste.groupby(['kreisnummer'])['Bruttoleistung'].sum().reset_index()

# Convert 'kreisnummer' column to integer type
kreisliste['kreisnummer'] = kreisliste['kreisnummer'].astype(int)

# Dictionary mapping kreisnummer to Bundesland names
kreis_bundesland_mapping = {
    1: 'Schleswig Holstein',
    2: 'Hamburg',
    3: 'Lower Saxony',
    4: 'Bremen',
    5: 'North-Rhine-Westphalia',
    6: 'Hesse',
    7: 'Rhineland Palatinate',
    8: 'Baden-Württemberg',
    9: 'Bayern',
    10: 'Saarland',
    11: 'Berlin',
    12: 'Brandenburg',
    13: 'Mecklenburg-Vorpommern',
    14: 'Saxony',
    15: 'Saxony-Anhalt',
    16: 'Thuringia'
}

# Check unique values in the 'kreisnummer' column
unique_kreisnummern = kreisliste['kreisnummer'].unique()
print("Unique Kreisnummern:", unique_kreisnummern)

# Rename 'Kreisnummer' column to 'Bundesland Number'
sum_bruttoleistung_per_kreisnummer.rename(columns={'kreisnummer': 'Bundesland Number'}, inplace=True)

# Add Bundesland column based on kreisnummer
sum_bruttoleistung_per_kreisnummer['Bundesland'] = sum_bruttoleistung_per_kreisnummer['Bundesland Number'].map(kreis_bundesland_mapping)

# Setting display format for float values
pd.options.display.float_format = '{:.0f}'.format

# Display the DataFrame
display(sum_bruttoleistung_per_kreisnummer)

Unique Kreisnummern: [ 1  3  5  8 12  6 15 16 13 14  7  9 10  4]


,Bundesland Number,Bruttoleistung,Bundesland
0,1,1062763,Schleswig Holstein
1,3,1483388,Lower Saxony
2,4,15,Bremen
3,5,1459398,North-Rhine-Westphalia
4,6,238652,Hesse
5,7,247541,Rhineland Palatinate
6,8,229848,Baden-Württemberg
7,9,81203,Bayern
8,10,25850,Saarland
9,12,828382,Brandenburg
